In [1]:
import pandas as pd
import numpy as np

In [2]:
# Dataset description:
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20for%20Individual%20Contribution%20Data.htm
columns_campfin22_indivs22 = ['cycle', 'fectransid', 'contribid', 'contrib_last', 'contrib_first', 'recipid', 'orgname', 
                    'ultorg', 'realcode', 'date', 'amount', 'street', 'city', 'state',
                    'zip', 'recipcode', 'type', 'cmteid', 'otherid', 'gender', 'microfilm',
                    'occupation', 'employer', 'source']
columns_campfin22_indivs22 = dict(enumerate(columns_campfin22_indivs22))

In [3]:
# This dataset is huge, and crashes my computer. For now, limiting to 1000000 rows
# until a better way is found to trim off this dataset to only the last cycle or two.
df = pd.read_csv('../../data/open_secrets/CampaignFin22/indivs22.csv', nrows=1000000, on_bad_lines='skip', sep=',', header=None, na_values=['N/A', 'NA'], encoding='ISO-8859-1')
df = df.rename(columns=columns_campfin22_indivs22)

/var/folders/rm/kxyn59hn41j35zkp54p83xp40000gn/T/ipykernel_86986/3794931839.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/open_secrets/CampaignFin22/indivs22.csv', nrows=1000000, on_bad_lines='skip', sep=',', header=None, na_values=['N/A', 'NA'], encoding='ISO-8859-1')


In [4]:
# Get rid of the pipes on both sides of the data.
df = df.applymap(lambda x: x.replace('|', '') if type(x) == str else x)

# Other cleanup.
df['cycle'] = df['cycle'].apply(lambda x: int(x))
df['fectransid'] = df['fectransid'].apply(lambda x: int(x))
df['date'] = df['date'][:9]
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
# df['microfilm'] = df['microfilm'].fillna(0)
# df['microfilm'] = df['microfilm'].apply(lambda x: 0 if x == '' else int(x))

In [5]:
pd.options.display.max_columns = None
df

,cycle,fectransid,contribid,contrib_last,contrib_first,recipid,orgname,ultorg,realcode,date,amount,street,city,state,zip,recipcode,type,cmteid,otherid,gender,microfilm,occupation,employer,source
0,2022,4061520221505841534,r0014256510,DILLARD,DANIEL,C00000935,[24T Contribution],,Z9500,2021-08-29,35,,HIDALGO,TX,78557,DP,24T,C00401224,C00000935,M,202205149504021121,NOT EMPLOYED,NOT EMPLOYED,Rept
1,2022,4061520221505841535,r0015503614,WHITE,SCOTTO,C00633404,[24T Contribution],,Z9500,2021-08-29,15,,YONKERS,NY,10701,PI,24T,C00401224,C00633404,M,202205149507911003,NOT EMPLOYED,NOT EMPLOYED,Rept
2,2022,4061520221505841536,p0003861308,DOMINGUEZ,SALLIE,C00632398,[24T Contribution],,Z9500,2021-08-29,10,,CAMPBELL,CA,95011,PI,24T,C00401224,C00632398,F,202205149504046504,HEALTHCARE,MCDONALD FAMILY DENTAL,Rept
3,2022,4061520221505841543,p0003689535,BRADLEY,JANNETTE,C00678839,[24T Contribution],,Z9500,2021-08-29,26,,OAK LAWN,IL,60453,PI,24T,C00401224,C00678839,F,202205149503347140,LIBRARIAN,MEDICAL CENTER,Rept
4,2022,4061520221505841545,q0002181378,BHATT,TANYA,C00580068,[24T Contribution],,Z9500,2021-08-29,50,,MIAMI BEACH,FL,33141,PI,24T,C00401224,C00580068,F,202205149503225635,MKTG,FRYE FINANCIAL,Rept
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2022,4021620231729136950,p0003928529,MOORE,LISA MIA,C00401224,,,J1200,NaT,1,,GLENDALE,CA,91202,PI,15,C00401224,NaN,F,202301319576977108,CLINICAL LAB SCIENTIST,CEDARS SINAI MEDICAL CENTER,P/PAC
999996,2022,4021620231729136951,p0003928529,MOORE,LISA MIA,C00401224,,,J1200,NaT,1,,GLENDALE,CA,91202,PI,15,C00401224,NaN,F,202301319576977109,CLINICAL LAB SCIENTIST,CEDARS SINAI MEDICAL CENTER,P/PAC
999997,2022,4021620231729136958,p0003928529,MOORE,LISA MIA,C00401224,,,J1200,NaT,1,,GLENDALE,CA,91202,PI,15,C00401224,NaN,F,202301319576977111,CLINICAL LAB SCIENTIST,CEDARS SINAI MEDICAL CENTER,P/PAC
999998,2022,4021620231729136959,p0003928529,MOORE,LISA MIA,C00401224,,,J1200,NaT,1,,GLENDALE,CA,91202,PI,15,C00401224,NaN,F,202301319576977111,CLINICAL LAB SCIENTIST,CEDARS SINAI MEDICAL CENTER,P/PAC


In [6]:
# Donor Albert Allens'recipient is Andre Carlson, via Act Blue (did Allen specify Andre, or did ActBlue specify it?)
# Andre's cid is shown here as the recipid field.
# otherid "C00442921" is his committee.
# CmteID "C00401224" is ActBlue (cmtes527.csv)
df[df['recipid'] == 'N00029513']

,cycle,fectransid,contribid,contrib_last,contrib_first,recipid,orgname,ultorg,realcode,date,amount,street,city,state,zip,recipcode,type,cmteid,otherid,gender,microfilm,occupation,employer,source
973145,2022,4021620231727375943,f1100207706,ALLEN,ALBERT,N00029513,[24T Contribution],,Z9500,NaT,25,,INDIANAPOLIS,IN,46228,DW,24T,C00401224,C00442921,M,202301319576527439,PHYSICIAN SCIENTIST,ELI LILLY,Rept


In [7]:
# No individual made a donation that got funneled Goldberg. (?)
df[df['recipid'] == 'N00036203']

,cycle,fectransid,contribid,contrib_last,contrib_first,recipid,orgname,ultorg,realcode,date,amount,street,city,state,zip,recipcode,type,cmteid,otherid,gender,microfilm,occupation,employer,source
